# Importing Libaries

In [2]:
# %pip install azureml-core
from azureml.core.authentication import InteractiveLoginAuthentication
auth = InteractiveLoginAuthentication()
from operator import itemgetter
import json
import os
import requests
import azureml.core
from azureml.core import Workspace
from azureml.core import Datastore
from azureml.core import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Model
from azureml.core import Environment, Experiment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.compute import AksCompute
from azureml.core.model import InferenceConfig
from azureml.core.runconfig import RunConfiguration
from azureml.data.data_reference import DataReference

print("Ready to use Azure ML", azureml.core.VERSION)


config_file_path = './config.json'
error_type_message = 'Error type:'
json_loads_error_message = 'json.loads() ValueError for JSON object:'

# This setup_config.json file establish session based on custom config
try:
    with open(config_file_path) as setup_config_data:
        setup_config = json.load(setup_config_data)
except ValueError as error:
    print(f"{error_type_message} {type(error)}")
    print(f"{json_loads_error_message} {error}")

subscription_id = setup_config['subscription_id']
resource_group = setup_config['resource_group']
workspace_name = setup_config['workspace_name']

ws = Workspace(
    subscription_id=subscription_id,
    resource_group=resource_group,
    workspace_name=workspace_name
)

print(ws.name, "loaded")
keyvault = ws.get_default_keyvault()
# ws.get_details()
print(
    'Ready to use Azure ML {} to work with {}'.format(
        azureml.core.VERSION,
        ws.name))

Ready to use Azure ML 1.56.0
azureml_demo loaded
Ready to use Azure ML 1.56.0 to work with azureml_demo


In [3]:
import os
# Create a folder for the pipeline step files
experiment_folder = 'test_pip_MSAI'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

test_pip_MSAI


In [4]:
%%writefile $experiment_folder/Training.py
from azure.identity import ManagedIdentityCredential, DefaultAzureCredential
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azureml.core import Run
import re
from datetime import datetime
import pdfplumber
import os
import pandas as pd
import openai
import chardet
from decimal import Decimal, InvalidOperation
import glob
import azure.ai.formrecognizer
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import os
import pickle
import numpy as np
np.int = np.int32
np.float = np.float64
np.bool = np.bool_
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import time
import json
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
import fitz
# import PyMuPDF
from dateutil import parser
from datetime import datetime
import pytz
import ast
import multiprocessing
import os
import openai
from openai import AzureOpenAI
import langdetect
from langdetect import detect
import ast


LOCAL_RUN = 'N'

pd.set_option('mode.chained_assignment', None)

if LOCAL_RUN == 'N':
    # Get the experiment run contexty
    run = Run.get_context()
    ws = run.experiment.workspace
else:
    # Load the workspace from the saved config file
    ws = Workspace.from_config()


print('Ready to work with {}'.format(ws.name))

keyvault = ws.get_default_keyvault()

identity_client_id = keyvault.get_secret("clientid")

credential = ManagedIdentityCredential(client_id=identity_client_id)
print("credential", credential)

openai.api_type = "azure"
openai.api_base = "https://openai-finops-np-002.openai.azure.com/"
openai.api_version = "2024-02-15-preview"
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
 
client = AzureOpenAI(
  azure_endpoint = openai.api_base, 
  azure_ad_token_provider=token_provider, 
  api_version=openai.api_version
) 

start_time = time.time()

def get_completion(prompt):
    response = client.chat.completions.create(
        model="openainpfinos02",
        seed=42,
        response_format={ "type": "json_object" },
        messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content":prompt}])
    response = response.choices[0].message.content
    return response  


def form_reading_pdf(filename, endpoint, credential):
    print("inside form recognizer")
    client = DocumentAnalysisClient(endpoint, credential = credential)
 
    # Use the custom model to extract information from the document
    with open(filename, "rb") as f:
        poller = client.begin_analyze_document("prebuilt-layout", f.read())
        result = poller.result()
 
    output = {}
    page_number = 1  # Initialize page number
    for page in result.pages:
        text = ""
        for line in page.lines:
            text += line.content + "\n"
        output['Page number ' + str(page_number)] = text.strip()
        page_number += 1  # Increment page number
 
    return json.dumps(output)


def form_convert_dict_to_list_of_dicts(input_string):
    input_dict = ast.literal_eval(input_string)
    list_of_dicts = []
    for key, value in input_dict.items():
        list_of_dicts.append({key: value})
    return list_of_dicts

def convert_dict_to_list_of_dicts(input_dict):
    # input_dict = ast.literal_eval(input_string)
    list_of_dicts = []
    for key, value in input_dict.items():
        list_of_dicts.append({key: value})
    return list_of_dicts

def reading_pdf(filename, endpoint, credential):
    print("inside reading pdf func",filename)
    doc = fitz.open(filename)
    page = doc[0]
    if len(page.get_text())>100:
        print("********pdf is editable*********",len(page.get_text()))
        
    # print(len(doc))
        final_dict = {}
        for i in range(len(doc)):
            page = doc[i]  # The third page is at index 2 
            page_content = page.get_text()
            page_number_key = f"Page number {i+1}"
            final_dict[page_number_key] = page_content
            final_dict_1 = convert_dict_to_list_of_dicts(final_dict)
        return final_dict_1

    else:
        print("**************pdf is not editable*************")
        final_dict = form_reading_pdf(filename, endpoint, credential)
        final_dict_1 = form_convert_dict_to_list_of_dicts(final_dict)
        return final_dict_1
    
model1 = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

## Compute embeddings
def get_embeddings(text, output_path_emb):
    print("inside embeddings")
    embeddings = model1.encode(text)
    with open(output_path_emb, 'wb') as f:
        pickle.dump(embeddings, f)
    # return embeddings
    
def load_embeddings_from_pickle(embeddings_path):
    print("Inside load embedding func")
    with open(embeddings_path, 'rb') as f:
        embeddings = pickle.load(f)
    return embeddings

def save_list_to_file(pdf_text, output_path):
    print("inside save_list_to_file")
    pdf_text = str(pdf_text)
    with open(output_path, "w") as file:
        file.write(pdf_text)
        
def read_list_from_file(filename):
    print("inside read_list_from_file")
    with open(filename, 'r') as file:
        return file.read()

def find_closely_related_chunks(query, embeddings, text, threshold=0.0, top_k=15):
    similarity_scores = cosine_similarity([query], embeddings)[0]
    above_threshold_indices = np.where(similarity_scores > threshold)[0]
    top_k_indices = above_threshold_indices[np.argsort(similarity_scores[above_threshold_indices])[::-1][:top_k]]
    
    print("Top 3 similarity scores:",format(top_k))
    for idx in top_k_indices:
        print("Index {}: {}".format(idx, similarity_scores[idx]))
    
    closely_related_chunks = []
    for idx in top_k_indices:
        chunk_key = list(text[idx].keys())[0]
        similarity_score = similarity_scores[idx]
        closely_related_chunks.append({f"{chunk_key}, similarity_score: {similarity_score:.8f}": text[idx][chunk_key]})
    
    return closely_related_chunks

def create_chunks(list_of_dicts,chunk_size=1):
    # print("list_of_dicts-------------",len(list_of_dicts))
    # print("list_of_dicts-------------",list_of_dicts)
    chunks = []
    for i in range(0, len(list_of_dicts), chunk_size):
        chunk = list_of_dicts[i:i+chunk_size]
        chunks.append(chunk)
    return chunks

def summary1(contract):
    prompt = f"""
    You will be provided a contract details {contract}. You need to compose a brief description of the contract, specifying the parties involved as the purchaser, the supplier, and affliates and clarify their roles. Display the response in JSON format."""
    response = get_completion(prompt)
    # print('response---------------------------------------',response)
    return response
 
def extract_text_from_image(image_bytes, endpoint, credential):
    # Initialize DocumentAnalysisClient
    client = DocumentAnalysisClient(endpoint, credential = credential)
 
    # Call Azure Form Recognizer to extract text from the image
    poller = client.begin_analyze_document("prebuilt-layout", image_bytes)
    result = poller.result()
 
    # Extract text from the result
    text = ""
    for page in result.pages:
        for line in page.lines:
            text += line.content + "\n"
 
    return text.strip()
 
def extract_text_from_pdf(pdf_path, endpoint, credential):
    print("Extracting text from PDF...")
    output = []
    output1 = []
    # Initialize DocumentAnalysisClient
    client = DocumentAnalysisClient(endpoint, credential = credential)
 
    # Open the PDF file
    with fitz.open(pdf_path) as doc:
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            image_list = page.get_images(full=True)

            if image_list:
                print('image')
                # Extract text from the image using Azure Form Recognizer
                image_bytes = page.get_pixmap().tobytes()
                extracted_text = extract_text_from_image(image_bytes, endpoint, credential)
                output1.append({f"Page number {page_num+1}": extracted_text})
            else:
                text = page.get_text()            
                # print('editable')
                output.append({f"Page number {page_num+1}": text.strip()})
 
    return output1

def format_response(text, response):
    
    # Step 1: Extract clause reference value
    clause_reference = next((value for key, value in response.items() if 'clause' in key.lower()), None)

    # Step 2: Automatically remove keys containing the word "clause"
    keys_to_remove = [key for key in response if 'clause' in key.lower()]
    for key in keys_to_remove:
        response.pop(key, None)
    # print('updated response---------------',response)

    # Step 3: Extract page number and similarity score
    page_number = ""
    similarity_score = ""
    for item in text:
        for key in item.keys():
            match = re.search(r'Page number (\d+), similarity_score: (\d+\.\d+)', key)
            if match:
                page_number = match.group(1)
                similarity_score = float(match.group(2))

    # Step 4: Determine rank
    if similarity_score > 0.3:
        rank = "Rank High"
    elif 0.2 <= similarity_score <= 0.3:
        rank = "Rank Medium"
    else:
        rank = "Rank Low"

    # Step 4: Update response dictionary
    for key in response:
        response[key] = f"{response[key]}:{clause_reference}:{rank}:Page {page_number}:score {similarity_score:.2f}"
    
    return response
    # print('new response------------------------------',response)
    
    
def map_keys_by_order(data, new_keys):
    if len(data) != len(new_keys):
        raise ValueError("Number of new keys must match the number of keys in the dictionary.")

    updated_dict = {}
    for i, (key, value) in enumerate(data.items()):
        updated_dict[new_keys[i]] = value

    return updated_dict
    
def is_numeric(val):
    try:
        int(val)
        return True
    except ValueError:
        return False

def process_payment_terms(data):
    payment_terms = data.get('PaymentTerms', '').lower()
    payment_terms_in_days = data.get('PaymentTermsInDays', '')

    if payment_terms == 'numeric':
        if is_numeric(payment_terms_in_days):
            return data
        else:
            for key in data:
                data[key] = 'na'
            return data
    else:
        return data
    
    
def process_payment_clause(data):
    payment_terms_ref = data.get('Payment Terms_Clause Reference', '').lower()
    payment_terms = data.get('PaymentTerms', '').lower()
    payment_terms_in_days = data.get('PaymentTermsInDays', '').lower()

    if payment_terms_ref != 'na' and payment_terms == 'na' and payment_terms_in_days == 'na':
        for key in data:
            data[key] = 'na'

    return data

def process_dict(d, keys, valid_values):

    """

    Converts dictionary values to uppercase and replaces them if they match valid_values.
 
    Args:

        d (dict): The dictionary containing the values to be processed.

        valid_values (list): List of valid values.
 
    Returns:

        dict: The modified dictionary.

    """

    # Convert valid_values to lowercase
    lower_valid_values = [value.lower() for value in valid_values]
 
    # Convert dictionary values to lowercase and replace if valid

    for key, value in d.items():
        if key == keys:
            # print(keys)
            if isinstance(value, str):
                # print(value)
                lower_value = value.lower()
                # print("lower_value", lower_value)
                if lower_value in lower_valid_values:
                    d[key] = valid_values[lower_valid_values.index(lower_value)]
                    # print(d)
                else:
                    d[key] = 'na'
 
    return d

def convert_date_format(response, key):
    if key in response:
        value = response[key]
        if not isinstance(value, str):
            response[key] = 'na'
            return response
        try:
            parsed_date = parser.parse(value)
            formatted_date = parsed_date.strftime('%m/%d/%Y')
            response[key] = formatted_date
        except ValueError:
            response[key] = 'na'
    return response

def convert_pipe_to_space(dictionary):
    # Iterate over each key-value pair in the dictionary
    for key, value in dictionary.items():
        # If the value is a string and contains '|'
        if isinstance(value, str) and '|' in value:
            # Replace '|' with space
            dictionary[key] = value.replace('|', ' ')
    return dictionary

def process_clause_section(response):
    for key in response.keys():
        if 'clause' in key.lower():
            response[key] = ''.join(ch if ch.isalnum() or ch in ['.', '-'] else ' ' for ch in response[key])
    return response

def remove_no_na_response(response):
    # Get the first key of the dictionary
    first_key = list(response.keys())[0]

    if response[first_key].lower() == 'no':
        for key in response.keys():
            if 'clause' in key.lower():
                if response[key].lower() == 'na':
                    for key in response.keys():
                        response[key] = 'na'
                        
    return response

def get_details_Payment_terms(chunks):
     
    l = []
    for text in chunks:
        keywords = ['Payment terms','Payment Terms in days','Payment Terms_Clause Reference']
        

        output_format = """
            {"<Payment terms>":"<Identified answer>",
            "<Payment Terms in days>":"<Identified answer>",           
            "<Payment Terms_Clause Reference>":"<Identified answer>"}"""
                
        
        prompt = f"""
        You are a master service agreement (MSA) details provider AI assistance.\n
        Your task is you will be provided a list of keywords {keywords} you need to extract the relevant answer from the MSA Agreement.\n
        You will be provided a top keyword matching pages from the MSA Agreement delimited by triple backticks.\n
        Do not provide extra explantion to your answer. If you are unable to extract the answer, then leave it as 'na'. Do strictly follow the below instruction strictly.\n
        
        Instruction:\n
 
        Step1: Understand the contract and locate the 'Payment terms'. Based on the context, set 'payment terms' to either 'Numeric' or if statement of work is mentioned, then set it to 'As specified by SOW'. Other than these two options, don't provide response to payment terms..\n
        Step2: If 'Payment terms' is 'Numeric' then extract the payment terms in days value and set it to 'Payment Terms in days' elif 'Payment terms' is not 'numeric' then set 'Payment Terms in days' to 'na'.\n 
        Step3: If the value of 'payment terms' is either **Numeric** or **As specified by SOW**, you extract the corresponding clause/subclause/Appendix/Execution/Annex/Exhibit and assign it to 'Payment Terms_Clause Reference' where the payment terms are mentioned. If you unable to extract 'Payment Terms_Clause Reference' then leave it as 'na'.
        'Payment terms_Clause Reference' this value should be in the form of: 'Sec <section number>'.
        Step4: Display the response strictly in a JSON format:{output_format}. 

        MSA Agreement : ```{text}```
        keywords: {keywords}
        """
        for _ in range(1):
            
            try:

                response = get_completion(prompt)
                response = ast.literal_eval(response)
                print('response------------------------------', response)

                #changing the key name in response
                new_keys= ['PaymentTerms','PaymentTermsInDays','Payment Terms_Clause Reference']
                response = map_keys_by_order(response,new_keys)
                response = convert_pipe_to_space(response)
                response = process_clause_section(response)
                #validation
                valid_values = ['Numeric', 'As specified in SOW', 'na']
                response = process_dict(response, 'PaymentTerms', valid_values)

                response = process_payment_terms(response)  #checking numeric - days in integer 
                print('response checking numeric int:',response)

                response = process_payment_clause(response)
                print('response checking clause :',response)
                
                response = remove_no_na_response(response)
                print('removing No and na response-------------',response)

                #If all values are na then not considering
                if all(value.lower() != 'na' for value in response.values()):
                    print("Not all values are 'na'. Proceeding to the next text.")
                    # continue

                    response = format_response(text,response)
                    print('formated response------------------------------',response)

                    l.append(response)
                break
            except Exception as e:
                if '429' in str(e):  # If error is 429
                    print("429 error encountered. Waiting for 4 seconds before retrying.")
                    time.sleep(4)  # Wait for 4 seconds before retrying
                else:
                    print(f"Error processing text: {e}")
                    break  # If error is not 429, break the retry loop and continue with the next text
    return l
        
    

Writing test_pip_MSAI/Training.py


In [2]:
def detect_languages(text):
    try:
        lang = detect(text)
    except:
        return 'Unable to detect language'
    
    if lang == 'en':
        return 'english'
    else:
        return 'Unable to detect language'

def read_pdf_and_detect_language(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    
    language = detect_languages(text)
    return language

def process(filename):

    def keywords_embeddings(keywords, filename):
        print("inside keywords_embeddings")
        output_folder = "keyword_embeddings/"
        os.makedirs(os.path.dirname(output_folder),exist_ok = True)
        keywords_1 = keywords.replace(" ","_")
        output_path = os.path.join(output_folder, keywords_1 + "_" + filename.replace(".PDF",".pkl"))
        return output_path

    keywords1 = 'payment terms'
    output_path1 = keywords_embeddings(keywords1, filename)
    keywords_embeddings_1 = get_embeddings(keywords1,output_path1)

    return None

def file_failure_status(filename,message):
    
    df = pd.DataFrame()

    cols = ['PaymentTerms','PaymentTermsInDays']

    for col in cols:
        if col not in df.columns:
            df[col] = 'nan'

    for col in df.columns:

        # Filter out NaN values
        df_cleaned = df.dropna()
        
        # Convert cleaned DataFrame to dictionary with index numbering keys
        json_data = {}
        for col in df_cleaned.columns:
            json_data[col] = [{str(i): v for i, v in enumerate(df_cleaned[col])}]

    # Prepare the result
    result = {"data": [json_data]}

    # Get the current time in IST

    ist_timezone = pytz.timezone("Asia/Kolkata")

    current_time_ist = datetime.now(ist_timezone)

    extractiontime = current_time_ist.strftime("%Y-%m-%d %H:%M:%S")

    print("Current IST time:", current_time_ist.strftime("%Y-%m-%d %H:%M:%S"))

    # Add a new key-value pair
    result1 = {}

    # Split the filename into name and extension
    name, extension = os.path.splitext(filename)

    # Replace the .pdf extension with .json
    json_filename = name 

    result1["AIExtractionStatus"] = "Failure"
    
    result1["Message"] = message

    result1["UniqueID"]=json_filename

    result1["AIExtractionDateTime"]= extractiontime

    result1.update(result)

    print(json.dumps(result1, indent=4))

    # Create the folder if it doesn't exist
    os.makedirs(local_outputfolder, exist_ok=True)

    # Full path to the file
    output_file_path = os.path.join(local_outputfolder, json_filename + '.json')

    # Write the JSON object to the file
    with open(output_file_path, 'w') as f:
        json.dump(result1, f, indent=4)

    print(f"JSON has been written to {output_file_path}")
    return filename
    
    
def extraction(filename):
    print('**************inside extraction functions***************')
    print('filename-------------',filename)
    try:
        #Reading the pdf text from output_folder2
        output_folder2= "pdf_text/"
        output_path2 = os.path.join(output_folder2,filename.replace(".PDF",".txt"))
        print('Reading pdf text inside extraction')
        pdf_text_1 = read_list_from_file(output_path2)
        pdf_text = ast.literal_eval(pdf_text_1)

        # Reading the summary text from output_folder3
        output_folder3= "summary_text/"
        output_path3 = os.path.join(output_folder3,filename.replace(".PDF",".txt"))
        print('reading summary text inside extraction')
        summary_text = read_list_from_file(output_path3)
        summary_text1 = ast.literal_eval(summary_text)
        summary = summary1(summary_text1)

        def keywords_path(keywords,filename):
            output_folder = "keyword_embeddings/"
            keywords1 = keywords.replace(" ","_")
            output_path = os.path.join(output_folder,keywords1 + "_" + filename.replace(".PDF",".pkl"))
            return output_path

        #loading pdf text embedding from new_embedding folder
        output_folder= "new_embedding/"
        output_path = os.path.join(output_folder,filename.replace(".PDF",".pkl"))
        embeddings = load_embeddings_from_pickle(output_path)

        
        try:
            keywords2 = 'payment terms'
            output_path2 = keywords_path(keywords2, filename)
            keywords_embeddings2 = load_embeddings_from_pickle(output_path2)
            similar_chunks2 = find_closely_related_chunks(keywords_embeddings2, embeddings, pdf_text)
            top_3_similar_chunks2 = create_chunks(similar_chunks2,chunk_size=1)
            print('length of chunk size: ',len(top_3_similar_chunks2))
            output2 = get_details_Payment_terms(top_3_similar_chunks2)
        except Exception as e:
            output2 = []
            print(f"An error occurred: {e}")
            
        out = output2

        combined_data = {}
        for d in out:
            for key, value in d.items():
                if key in combined_data:
                    combined_data[key].append(value)
                else:
                    combined_data[key] = [value]

        if combined_data:
            try:
                # Create a DataFrame for each key-value pair and concatenate them
                df = pd.concat({k: pd.DataFrame(v) for k, v in combined_data.items()}, axis=1)

                # Drop the first level of the column names
                df.columns = df.columns.droplevel(1)

                df = df.rename(columns={'ForceMajeure': 'ForceMajeureClause', 'ForceMajeureLinkToOther': 'ForceMajeureLinkToOtherClauses'})

                cols = ['PaymentTerms','PaymentTermsInDays']

                for col in cols:
                    if col not in df.columns:
                        df[col] = 'nan'

                for col in df.columns:
                    df[col] = df[col].astype(str).str.replace(':', '|')

                    # Filter out NaN values
                    df_cleaned = df.dropna()


                    # Convert cleaned DataFrame to dictionary with index numbering keys
                    json_data = {}
                    for col in df_cleaned.columns:
                        json_data[col] = [{str(i): v for i, v in enumerate(df_cleaned[col])}]

                # Prepare the result
                result = {"data": [json_data]}

                # Iterate over the dictionary
                for item in result['data']:
                    for sub_item in item.values():
                        for dict_item in sub_item:
                            for key in list(dict_item.keys()):  # Use list to create a copy of keys
                                value = dict_item[key]
                                if value.split('|')[0] in ['na', 'Not mentioned', 'Not defined', 'Not Mentioned', 'Not Defined']:
                                    del dict_item[key]  # Remove the pair from the dictionary

                def remove_nan_pairs_and_reset_index(obj):
                    if isinstance(obj, dict):
                        new_obj = {k: remove_nan_pairs_and_reset_index(v) for k, v in obj.items() if v != 'nan'}
                        if all(k.isdigit() for k in new_obj.keys()):
                            new_obj = {str(i): v for i, (k, v) in enumerate(new_obj.items())}
                        return new_obj
                    elif isinstance(obj, list):
                        return [remove_nan_pairs_and_reset_index(item) for item in obj]
                    else:
                        return obj


                # Remove 'nan' pairs
                filtered_data = remove_nan_pairs_and_reset_index(result)

                # Get the current time in IST

                ist_timezone = pytz.timezone("Asia/Kolkata")

                current_time_ist = datetime.now(ist_timezone)

                extractiontime = current_time_ist.strftime("%Y-%m-%d %H:%M:%S")

                print("Current IST time:", current_time_ist.strftime("%Y-%m-%d %H:%M:%S"))

                # Add a new key-value pair
                result1 = {}

                # Split the filename into name and extension
                name, extension = os.path.splitext(filename)

                # Replace the .pdf extension with .json
                # json_filename = name + '.json'
                json_filename = name 

                result1["AIExtractionStatus"] = "Success"

                result1["UniqueID"]=json_filename

                result1["AIExtractionDateTime"]= extractiontime

                result1.update(filtered_data)

                print(json.dumps(result1, indent=4))

                # Create the folder if it doesn't exist
                os.makedirs(local_outputfolder, exist_ok=True)

                # Full path to the file
                output_file_path = os.path.join(local_outputfolder, filename.replace(".PDF",".json"))

                # Write the JSON object to the file
                with open(output_file_path, 'w') as f:
                    json.dump(result1, f, indent=4)

                print(f"JSON has been written to {output_file_path}")

                return filename

            except ValueError as e:
                print(f"Error occurred while creating DataFrame: {e}")
                message = "Error occurred while creating DataFrame"
                result1 = file_failure_status(filename,message)
            
        else:
            print("GPT is not able to extract. All the fields are empty")
            message = "GPT is not able to extract. All the fields are empty"
            result1 = file_failure_status(filename,message)
            return filename
                
    except Exception as e:
        print(f"Expection occured while reading the files: {e}")
        message = "Expection occured"
        result1 = file_failure_status(filename,message)
    
    return filename

#-------------------------------------------------------------------------------------------------------
def list_files(directory):
    # print(os.listdir(directory))
    return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]


def upload_file_to_blob(blob_service_client, file_path, container_name, blob_name):
    try:
        # Create a blob client using the local file name as the name for the blob
        blob_client = blob_service_client.get_blob_client(container_name, blob_name)
        # Upload the created file
        with open(file_path, "rb") as data:
            blob_client.upload_blob(data, content_type="application/json",overwrite=True)
    except Exception as e:
        print("AzureError: ", str(e))

def upload_files_to_blob_folder(blob_service_client, local_outputfolder, container_name, blob_outputfolder):
    for filename in os.listdir(local_outputfolder):#folder_path
        local_path = os.path.join(local_outputfolder, filename)
        blob_name = os.path.join(blob_outputfolder, filename)
        print(f"Uploading {local_path} to {blob_name}")
        upload_file_to_blob(blob_service_client, local_path, container_name, blob_name)

account_name = 'azstorageaccnpfinoper'
container_name = '757'

identity_client_id = keyvault.get_secret("clientid")

### Authenticate using MSI
credential = ManagedIdentityCredential(client_id=identity_client_id)
print("credential", credential)

global local_inputfolder, local_pdf_text, local_summary_text, local_new_embedding, localinput_foldername, blob_outputfolder, blob_inputfolder, blob_preprocessedfolder, blob_processedfolder

local_inputfolder = "Input"
local_outputfolder = 'Output'

# local_pdf_text = "MSA_test/pdf_text"
# local_summary_text = "MSA_test/summary_text"
# local_new_embedding = "MSA_test/new_embedding"
localinput_foldername = "Input/MSA_test/Input"

blob_outputfolder = 'MSA_test/OUTPUT'

blob_inputfolder = 'MSA_test/Input'
blob_preprocessedfolder = 'MSA_test/Pre-Processed'
blob_processedfolder = 'MSA_test/Processed'

### Authenticate using MSI
# credential = ManagedIdentityCredential(client_id=identity_client_id)
# print("credential", credential)
# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=credential)
# Get the ContainerClient
container_client = blob_service_client.get_container_client(container_name)
# List all the blobs in the specific folder
blob_list = container_client.list_blobs(name_starts_with=blob_inputfolder)

## Checking Whether Output, pre-processed and Processed folders are avialable in Blob or not.
## If not then first create folders
blob_client_output = container_client.get_blob_client(blob=blob_outputfolder)
blob_client_processed = container_client.get_blob_client(blob=blob_processedfolder)
blob_client_preprocessed = container_client.get_blob_client(blob = blob_preprocessedfolder)
        
if not blob_client_output.exists():
    print(f"The folder '{blob_client_output}' does not exists.")
    blob_client_output.upload_blob(b"", overwrite=True)
    
else:
    print(f"The folder '{blob_client_output}' exist.")

    
if not blob_client_processed.exists():
    print(f"The folder '{blob_client_processed}' does not exists.")
    blob_client_processed.upload_blob(b"", overwrite=True)
    
else:
    print(f"The folder '{blob_client_processed}' exist.")

if not blob_client_preprocessed.exists():
    print(f"The folder '{blob_client_preprocessed}' does not exists.")
    blob_client_preprocessed.upload_blob(b"", overwrite=True)
    
else:
    print(f"The folder '{blob_client_preprocessed}' exist.")

# Check any files are available in Input Folder
if len(list(container_client.list_blobs(name_starts_with=blob_inputfolder))) != 0:
    print("Input Folder in not Empty. Processing files")
    ## Download all the blobs to your local
    for blob in blob_list:
        st_blob_down = time.time()
        print(f"\nDownloading the blob '{blob.name}' from the container to the local folder...'{local_inputfolder}/{blob.name}' ")
        os.makedirs(os.path.dirname(f"{local_inputfolder}/{blob.name}"), exist_ok=True)
        with open(f"{local_inputfolder}/{blob.name}", "wb") as data:
            blob_client = blob_service_client.get_blob_client(container_name, blob.name)
            download_stream = blob_client.download_blob()
            data.write(download_stream.readall())    
            

        ### List all the files present in Input folder 
    files = list_files(localinput_foldername)
    print("List of files::", files)
    end_blob_down = time.time()
    print('Time taken for downloading the files from blob',end_blob_down-st_blob_down)
          
    ###Moving all files from Input folder to Pre-Processed folder
    for blob in files:
        source_blob_path = f"{blob_inputfolder}/{blob}"
        print('source_blob_path', source_blob_path)
        destination_blob_path = f"{blob_preprocessedfolder}/{blob}"
        print('destination_blob_path', destination_blob_path)    
        source_blob_client = container_client.get_blob_client(source_blob_path)
        print('source_blob_client', source_blob_client)
        if source_blob_client.exists():
            destination_blob_client = container_client.get_blob_client(destination_blob_path)
            print('destination_blob_client', destination_blob_client)
            destination_blob_client.start_copy_from_url(source_blob_client.url)
            source_blob_client.delete_blob()
        else:
            print(f"Blob '{blob}' not found in the source container.")
            
    def main(filename):
        print('***********inside main function*************')
        result = []
        print("inside main func")
        for files in filename:
            print('files',files)
            start_ex = time.time()
            output = extraction(files)
            end_ex = time.time()
            print('Time taken for extraction',end_ex-start_ex)
            result.append(output)
        return result

    def is_english(text):
        try:
            language = detect(text)
        except:
            return "Unable to detect language"

        if language == 'en':
            return "English"
        else:
            return "Unable to detect language"

        
    def check_valid_files(files):
        result = []
        for file_path in files:
            print('file_path-----------',file_path)
            if file_path.endswith('.PDF'):
                result.append(file_path)
            else:
                message = 'File is not Pdf'
                output = file_failure_status(file_path,message)    
        return result
    
    def file_validation(file):
        print('******Inside file validation function************')
        result1 = []
        for filename in file:
            endpoint = "https://ctsazformrecpdfinops.cognitiveservices.azure.com/"
            #Reading pdf file from localinput_foldername
            file_path = os.path.join(localinput_foldername,filename) 
            start_re = time.time()
            output_json = reading_pdf(file_path, endpoint, credential)
            output_image_text = extract_text_from_pdf(file_path, endpoint, credential)
            
            output_json_dict = {list(d.keys())[0]: list(d.values())[0] for d in output_json}
            output_image_text_dict = {list(d.keys())[0]: list(d.values())[0] for d in output_image_text}
            # Find keys in output_json1 not present in output_json
            new_keys = {k: v for k, v in output_image_text_dict.items() if k not in output_json_dict}
            # Convert the dictionary to a list of dictionaries
            new_keys_list = [{k: v} for k, v in new_keys.items()]
    
            output_json.extend(new_keys_list)
            end_re = time.time()
            print('Time taken for reading pdf',end_re-start_re)
            pdf_text = output_json
            combined_text = " ".join([value for dictionary in pdf_text for value in dictionary.values()])
            # print('combined_text--------------------',combined_text)
            language = is_english(combined_text)
            first_page_text = list(pdf_text[0].values())[0]

            if "Get Adobe Reader" in first_page_text:
                print("PDF is not opened",filename)
                message = "PDF is not Opened"
                result = file_failure_status(filename,message) 

            elif "Unable to detect language" in language:
                print("PDF is not in english",filename)
                message = "PDF is not in English"
                result = file_failure_status(filename,message)  

            else:
                print('PDF can be processed',filename)
                #Saving the extracted text to output_folder2
                output_folder2= "pdf_text/"
                os.makedirs(os.path.dirname(output_folder2),exist_ok = True)
                output_path2 = os.path.join(output_folder2,filename.replace(".PDF",".txt"))
                plain_text = save_list_to_file(pdf_text, output_path2)

                output_summary = pdf_text[:10]

                #Saving the extracted text to output_folder2
                output_folder3= "summary_text/"
                os.makedirs(os.path.dirname(output_folder3),exist_ok = True)
                output_path3 = os.path.join(output_folder3,filename.replace(".PDF",".txt"))
                summary_text = save_list_to_file(output_summary, output_path3)

                #Embedding the pdf text and saving in output_folder
                output_folder= "new_embedding/"
                os.makedirs(os.path.dirname(output_folder),exist_ok = True)
                output_path_emb = os.path.join(output_folder,filename.replace(".PDF",".pkl"))
                embeddings_1= get_embeddings(pdf_text, output_path_emb)
                result1.append(filename)
        return result1

   
    # def final(path):
    st = time.time()
    print('***********print inside final function****')
    num_cores = multiprocessing.cpu_count()
    all_files = list_files(localinput_foldername)
    print("(1)all_files--------------",all_files)
    all_files1 = check_valid_files(all_files) # cheking the file type
    print('all valid files--------',all_files1) 
    files = file_validation(all_files1) 
    print("(2)files--------------",files)
    data_chunks = np.array_split(files, num_cores)
    print('data_chunks: ',data_chunks)
    print("(2)embeddings----------------------")
    se = time.time()
    embeddings = [process(file) for file in files]## creation of embedding for all the files
    ee = time.time()
    print("(3)embeddings completed--------------")
    sm = time.time()
    pool = multiprocessing.Pool(processes=num_cores)
    results = pool.map(main, data_chunks)
    print('(4)result step completed--------------------')
    # finaloutput = np.concatenate(results)
    em = time.time()
    # print("length of Final Output:::", len(finaloutput))
    # print("Final Output:::",finaloutput )
    print("Time taken for reading and embeddings pdf ",ee-se)
    print("Time taken for extraction ",em-sm)

    pool.close()
    pool.join()
    et = time.time()
    print('Total time taken ',et-st)

    os.makedirs(local_outputfolder, exist_ok=True)
    ###Uploding json back to Output folder of Blob
    if blob_service_client is not None:
        print("*********inside uploading**********")
        upload_files_to_blob_folder(blob_service_client, local_outputfolder, container_name, blob_outputfolder)
        ### Move the processed file to processed folder of blob
        for blob in all_files:
            print("blob------------",blob)
            source_blob_path = f"{blob_preprocessedfolder}/{blob}"
            print('source_blob_path', source_blob_path)
            destination_blob_path = f"{blob_processedfolder}/{blob}"
            print('destination_blob_path', destination_blob_path)    
            source_blob_client = container_client.get_blob_client(source_blob_path)
            print('source_blob_client', source_blob_client)
            if source_blob_client.exists():
                destination_blob_client = container_client.get_blob_client(destination_blob_path)
                print('destination_blob_client', destination_blob_client)
                destination_blob_client.start_copy_from_url(source_blob_client.url)
                source_blob_client.delete_blob()
            else:
                print(f"Blob '{blob}' not found in the source container.")
    else:
        print("Error in logging into blob service client.")
else:
    print('Input folder is empty')
    
    
end_time = time.time()
print('Time taken for running the whole script',end_time-start_time)

NameError: name 'keyvault' is not defined

In [ ]:
import azureml.core
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "MLNPFINOPSCOMPCLUSTER01"

try:
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Compute Cluster is not available')
    # If not, create it. The below statement will not run in non prod or prod due to lack of permission, only for lower env
    # compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D4_V3', max_nodes=3)
    # pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

pipeline_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration

# Create a Python environment for the experiment
my_env = Environment.from_conda_specification(name = "test_lang_MSA",
                                           file_path ='./MSAsdk.yml')

my_env.python.user_managed_dependencies = False                                    # Let Azure ML manage dependencies
my_env.docker.enabled = True                                                       # Use a docker container

# Register the environment (just in case previous lab wasn't completed)
my_env.register(workspace=ws)
registered_env = Environment.get(ws, 'test_lang_MSA')

In [ ]:
# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above. 
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env

print ("Run configuration created.")

In [ ]:
from azureml.pipeline.steps import PythonScriptStep
from azureml.core import Datastore, Dataset
from datetime import datetime

train_step_pd_1 = PythonScriptStep(name = "test_pipeline_MSATraining",
                                   script_name='Training.py',
                                   compute_target=pipeline_cluster,
                                   source_directory=experiment_folder,
                                   runconfig=pipeline_run_config,
                                   allow_reuse=False)
print("python_script_step created")

In [ ]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline

pipeline_steps = [train_step_pd_1]

pipeline = Pipeline(workspace = ws, steps=pipeline_steps)

print("Pipeline is built.")

In [ ]:
published_pipeline = pipeline.publish(name="test_pipeline_MSATraining", 
                                        description="NONProd Pipeline for MSA Extraction", 
                                        continue_on_step_failure=False)

In [ ]:
from azureml.pipeline.core import PublishedPipeline

pipeline_id = published_pipeline.id # use your published pipeline id
published_pipeline = PublishedPipeline.get(ws, pipeline_id)
published_pipeline

In [ ]:
# from azureml.core import Datastore, Dataset
# from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule, TimeZone

# # Submit the Pipeline every 5 minutes
# recurrence = ScheduleRecurrence(frequency="Minute", interval=60)
# experiment_name = "MSA_Prod"

# reactive_schedule = Schedule.create(ws, name="MSA_Prod_Pipeline", 
#                                     description="Prod Pipeline for MSA Extraction",
#                                     pipeline_id=pipeline_id, experiment_name=experiment_name, 
#                                     # datastore=datastore,
#                                    recurrence =recurrence)